In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
from sklearn.externals import joblib

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
path_temp = r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Processed_Datasets/bert_cls_embeddings_df.pkl"
f = open(path_temp,'rb')
bert_cls_embeddings_df = pickle.load(f)
f.close()

path_temp = r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Processed_Datasets/bert_cls_embeddings_text_df.pkl"
f = open(path_temp,'rb')
bert_cls_embeddings_text_df = pickle.load(f)
f.close()

path_temp = r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Processed_Datasets/bert_cls_embeddings_text_user_df.pkl"
f = open(path_temp,'rb')
bert_cls_embeddings_text_user_df = pickle.load(f)
f.close()

In [4]:
bert_cls_embeddings_df            = bert_cls_embeddings_df.drop(['tweet_id'],axis=1)
bert_cls_embeddings_text_df       = bert_cls_embeddings_text_df.drop(['tweet_id'],axis=1)
bert_cls_embeddings_text_user_df  = bert_cls_embeddings_text_user_df.drop(['tweet_id'],axis=1)

In [5]:
bert_cls_embeddings_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,label
0,-0.905121,0.696438,-0.410108,0.232647,0.543449,-0.533337,0.215387,0.233201,0.089233,-0.420655,0.481249,1.544584,0.434246,0.034868,-0.201352,0.066022,0.452728,0.379511,0.496514,-0.219640,-0.180710,-0.296065,0.480571,0.487984,0.728716,-0.109550,0.107091,-0.323230,-0.933372,0.477563,-0.386658,0.799414,0.182509,-0.596100,-0.086162,0.533405,-0.070791,-0.072786,0.265669,-0.151689,...,-0.430354,0.356484,-0.255353,-0.005549,-0.357581,1.386957,0.324273,-1.043557,-0.460201,-1.487272,-0.585807,0.135839,1.142539,-0.017003,-0.708275,0.278565,0.019340,0.310408,0.486976,0.527736,0.569209,0.080745,-0.674057,0.329639,0.675203,-1.046797,0.341848,-0.272043,0.130041,-0.783586,-1.073365,-0.338529,-0.804068,0.171743,-0.574692,-0.543318,0.156378,0.170637,-0.138473,0
1,0.689706,0.642599,0.444143,-0.156185,0.178482,0.593337,1.548959,0.618944,0.361017,-0.215186,-0.299446,0.462064,0.225799,0.785722,1.427248,0.715510,-0.412167,-0.448764,-0.537064,0.268391,-0.142524,0.031702,0.030639,-1.512518,-0.520966,0.045811,-0.085850,-0.505757,-1.523551,0.516662,0.286007,-1.117602,0.295372,-0.579215,-0.406015,0.751337,0.492198,-0.526615,0.092728,-0.619554,...,-0.492803,0.026878,0.724324,-1.071671,-0.171228,1.453391,0.453673,1.321678,-0.072233,-0.334832,0.932822,-0.727925,1.106685,0.509290,-0.375232,0.674632,-0.020029,0.157731,0.141453,0.056432,0.035102,0.056925,0.643272,0.402604,-0.608893,-0.401588,-0.362285,-0.346232,-0.032420,0.698916,0.634631,-0.253063,-1.021878,-1.244154,-0.878302,-0.240221,0.394787,-0.369758,0.030639,0
2,-0.061979,0.294637,1.001708,0.316195,-0.058020,-0.490087,1.557522,0.150213,0.149639,-0.383844,0.464099,0.774086,0.456768,0.599950,-0.803100,0.184758,0.659355,-0.154717,0.046750,-0.433777,-0.390224,-0.123316,0.467006,0.609863,0.751140,1.015003,0.431661,-0.088001,-0.743736,0.516859,-0.270634,0.150997,0.417484,-0.081443,0.233031,0.178677,0.007473,-0.515848,0.583279,-0.258523,...,-0.540568,-0.024013,0.596610,-0.141648,-0.815083,0.596107,0.211470,0.093223,-0.717144,-0.902454,-1.217862,1.452921,1.454406,-0.055318,-1.009279,0.773620,-0.308888,0.705804,-0.177382,0.314141,0.691915,0.332458,0.502953,0.352488,0.760131,0.158746,0.108935,0.175663,-0.192818,-0.230578,0.003267,-0.712366,-1.024835,0.269487,-1.143081,-0.806404,0.500546,-0.114903,-0.023173,0
3,-0.130727,0.846263,-0.153783,-0.298239,0.241699,0.418571,1.087748,0.833864,0.984728,-0.079688,1.077919,0.410899,0.490434,0.209261,-0.033841,0.542424,-0.025668,-0.603422,0.093339,-0.282455,-0.050540,-0.000507,-0.295670,-0.209408,0.232751,-0.052327,0.427314,0.102890,-1.486257,0.332552,-0.386459,0.570869,-0.372500,-0.042429,0.783949,0.707599,0.171860,-0.442963,0.202495,0.398543,...,-0.068244,0.687037,-0.098606,-0.198904,-0.279022,0.418850,0.799002,0.454156,-1.217718,-0.396879,0.044941,-0.049153,1.298311,0.036355,-1.654261,0.583082,0.021644,0.041548,0.565015,-0.202961,0.100423,0.568653,0.031493,0.762331,0.394372,0.265884,0.258600,-0.808737,-0.041802,-0.542989,-0.009147,0.045073,-0.252952,-0.832314,-0.809334,-0.415984,-0.047424,0.108040,-0.167144,0
4,0.065145,1.240352,1.072927,-0.018979,0.364369,0.343705,1.457816,0.543128,-0.314964,-0.346804,0.830394,0.301960,0.731687,0.547858,-0.223253,0.439696,0.509563,-0.173309,0.655493,-0.537015,-0.105235,-0.548334,0.872155,0.173772,0.550351,0.747159,0.115209,-0.144531,-1.432526,0.309350,0.079108,-0.327085,0.631277,-0.481930,1.251270,0.158709,-0.307430,-0.476765,0.208309,-0.681993,...,-0.728378,0.407879,-0.061552,-0.228083,-0.911799,0.439519,0.485681,0.741564,-0.540842,-1.116476,-1.391408,0.356337,1.248870,-0.163293,-1.291355,0.514375,0.226632,0.513620,0.062475,-0.634988,-0.363325,0.180359,-0.020946,0.040517,0.546352,-0.870934,-0.066124,-0.300122,-0.277932,0.109597,-0.746704,-0.611905,-1.246076,0.5985

In [6]:
bert_cls_embeddings_text_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,retweet_count,favorite_count,length_before,length_after,ratio_of_length,number_of_upper_chars,has_question_marks,number_of_question_marks,has_exclamations,number_of_exclamation_marks,label
0,-0.905121,0.696438,-0.410108,0.232647,0.543449,-0.533337,0.215387,0.233201,0.089233,-0.420655,0.481249,1.544584,0.434246,0.034868,-0.201352,0.066022,0.452728,0.379511,0.496514,-0.219640,-0.180710,-0.296065,0.480571,0.487984,0.728716,-0.109550,0.107091,-0.323230,-0.933372,0.477563,-0.386658,0.799414,0.182509,-0.596100,-0.086162,0.533405,-0.070791,-0.072786,0.265669,-0.151689,...,-0.585807,0.135839,1.142539,-0.017003,-0.708275,0.278565,0.019340,0.310408,0.486976,0.527736,0.569209,0.080745,-0.674057,0.329639,0.675203,-1.046797,0.341848,-0.272043,0.130041,-0.783586,-1.073365,-0.338529,-0.804068,0.171743,-0.574692,-0.543318,0.156378,0.170637,-0.138473,187425,954568,134,134,1.000000,2,0,0,0,0,0
1,0.689706,0.642599,0.444143,-0.156185,0.178482,0.593337,1.548959,0.618944,0.361017,-0.215186,-0.299446,0.462064,0.225799,0.785722,1.427248,0.715510,-0.412167,-0.448764,-0.537064,0.268391,-0.142524,0.031702,0.030639,-1.512518,-0.520966,0.045811,-0.085850,-0.505757,-1.523551,0.516662,0.286007,-1.117602,0.295372,-0.579215,-0.406015,0.751337,0.492198,-0.526615,0.092728,-0.619554,...,0.932822,-0.727925,1.106685,0.509290,-0.375232,0.674632,-0.020029,0.157731,0.141453,0.056432,0.035102,0.056925,0.643272,0.402604,-0.608893,-0.401588,-0.362285,-0.346232,-0.032420,0.698916,0.634631,-0.253063,-1.021878,-1.244154,-0.878302,-0.240221,0.394787,-0.369758,0.030639,333831,1505405,24,24,1.000000,0,0,0,0,0,0
2,-0.061979,0.294637,1.001708,0.316195,-0.058020,-0.490087,1.557522,0.150213,0.149639,-0.383844,0.464099,0.774086,0.456768,0.599950,-0.803100,0.184758,0.659355,-0.154717,0.046750,-0.433777,-0.390224,-0.123316,0.467006,0.609863,0.751140,1.015003,0.431661,-0.088001,-0.743736,0.516859,-0.270634,0.150997,0.417484,-0.081443,0.233031,0.178677,0.007473,-0.515848,0.583279,-0.258523,...,-1.217862,1.452921,1.454406,-0.055318,-1.009279,0.773620,-0.308888,0.705804,-0.177382,0.314141,0.691915,0.332458,0.502953,0.352488,0.760131,0.158746,0.108935,0.175663,-0.192818,-0.230578,0.003267,-0.712366,-1.024835,0.269487,-1.143081,-0.806404,0.500546,-0.114903,-0.023173,160323,660762,218,218,1.000000,7,1,1,1,1,0
3,-0.130727,0.846263,-0.153783,-0.298239,0.241699,0.418571,1.087748,0.833864,0.984728,-0.079688,1.077919,0.410899,0.490434,0.209261,-0.033841,0.542424,-0.025668,-0.603422,0.093339,-0.282455,-0.050540,-0.000507,-0.295670,-0.209408,0.232751,-0.052327,0.427314,0.102890,-1.486257,0.332552,-0.386459,0.570869,-0.372500,-0.042429,0.783949,0.707599,0.171860,-0.442963,0.202495,0.398543,...,0.044941,-0.049153,1.298311,0.036355,-1.654261,0.583082,0.021644,0.041548,0.565015,-0.202961,0.100423,0.568653,0.031493,0.762331,0.394372,0.265884,0.258600,-0.808737,-0.041802,-0.542989,-0.009147,0.045073,-0.252952,-0.832314,-0.809334,-0.415984,-0.047424,0.108040,-0.167144,154086,368883,72,72,1.000000,2,0,0,0,0,0
4,0.065145,1.240352,1.072927,-0.018979,0.364369,0.343705,1.457816,0.543128,-0.314964,-0.346804,0.830394,0.301960,0.731687,0.547858,-0.223253,0.439696,0.509563,-0.173309,0.655493,-0.537015,-0.105235,-0.548334,0.872155,0.173772,0.550351,0.747159,0.115209,-0.144531,-1.432526,0.309350,0.079108,-0.327085,0.631277,-0.481930,1.251270,0.158709,-0.307430,-0.476765,0.208309,-0.681993,...,-1.391408,0.356337,1.248870,-0.163293,-1.291355,0.514375,0.226632,0.513620,0.062475,-0.634988,-0.363325,0.180359,-0.020946,0.040517,0.546352,-0.870934,-0.066124,-0.300122,-0.277932,0.109597,-0.746704,-0.611905,-1.246076,0.598580,-0.580133,-0.914450,0.119558,-0.130259,0.039278,126506,445713,292,270,0.924658,7,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [7]:
bert_cls_embeddings_text_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,758,759,760,761,762,763,764,765,766,767,retweet_count,favorite_count,length_before,length_after,ratio_of_length,number_of_upper_chars,has_question_marks,number_of_question_marks,has_exclamations,number_of_exclamation_marks,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_life,verified,num_tweets,protected,posting_frequency,activity,avg_likes_per_tweet,follower_friends_ratio,label
0,-0.905121,0.696438,-0.410108,0.232647,0.543449,-0.533337,0.215387,0.233201,0.089233,-0.420655,0.481249,1.544584,0.434246,0.034868,-0.201352,0.066022,0.452728,0.379511,0.496514,-0.219640,-0.180710,-0.296065,0.480571,0.487984,0.728716,-0.109550,0.107091,-0.323230,-0.933372,0.477563,-0.386658,0.799414,0.182509,-0.596100,-0.086162,0.533405,-0.070791,-0.072786,0.265669,-0.151689,...,-0.783586,-1.073365,-0.338529,-0.804068,0.171743,-0.574692,-0.543318,0.156378,0.170637,-0.138473,187425,954568,134,134,1.000000,2,0,0,0,0,124,11,7,3,0,1,4726,852,43,17544,0,3978,0,9629,0,2.420563,1,1.821996,5.546948,0
1,0.689706,0.642599,0.444143,-0.156185,0.178482,0.593337,1.548959,0.618944,0.361017,-0.215186,-0.299446,0.462064,0.225799,0.785722,1.427248,0.715510,-0.412167,-0.448764,-0.537064,0.268391,-0.142524,0.031702,0.030639,-1.512518,-0.520966,0.045811,-0.085850,-0.505757,-1.523551,0.516662,0.286007,-1.117602,0.295372,-0.579215,-0.406015,0.751337,0.492198,-0.526615,0.092728,-0.619554,...,0.698916,0.634631,-0.253063,-1.021878,-1.244154,-0.878302,-0.240221,0.394787,-0.369758,0.030639,333831,1505405,24,24,1.000000,0,0,0,0,0,10,10,10,3,0,0,23729,11,9,4,0,256,0,235,0,0.917969,1,0.017021,2157.181818,0
2,-0.061979,0.294637,1.001708,0.316195,-0.058020,-0.490087,1.557522,0.150213,0.149639,-0.383844,0.464099,0.774086,0.456768,0.599950,-0.803100,0.184758,0.659355,-0.154717,0.046750,-0.433777,-0.390224,-0.123316,0.467006,0.609863,0.751140,1.015003,0.431661,-0.088001,-0.743736,0.516859,-0.270634,0.150997,0.417484,-0.081443,0.233031,0.178677,0.007473,-0.515848,0.583279,-0.258523,...,-0.230578,0.003267,-0.712366,-1.024835,0.269487,-1.143081,-0.806404,0.500546,-0.114903,-0.023173,160323,660762,218,218,1.000000,7,1,1,1,1,1,6,10,2,0,0,2826,2803,19,11789,1,2903,0,22287,0,7.677230,0,0.528963,1.008205,0
3,-0.130727,0.846263,-0.153783,-0.298239,0.241699,0.418571,1.087748,0.833864,0.984728,-0.079688,1.077919,0.410899,0.490434,0.209261,-0.033841,0.542424,-0.025668,-0.603422,0.093339,-0.282455,-0.050540,-0.000507,-0.295670,-0.209408,0.232751,-0.052327,0.427314,0.102890,-1.486257,0.332552,-0.386459,0.570869,-0.372500,-0.042429,0.783949,0.707599,0.171860,-0.442963,0.202495,0.398543,...,-0.542989,-0.009147,0.045073,-0.252952,-0.832314,-0.809334,-0.415984,-0.047424,0.108040,-0.167144,154086,368883,72,72,1.000000,2,0,0,0,0,14,13,11,3,0,0,1231,759,1,16627,0,2574,0,13653,0,5.304196,1,1.217828,1.621871,0
4,0.065145,1.240352,1.072927,-0.018979,0.364369,0.343705,1.457816,0.543128,-0.314964,-0.346804,0.830394,0.301960,0.731687,0.547858,-0.223253,0.439696,0.509563,-0.173309,0.655493,-0.537015,-0.105235,-0.548334,0.872155,0.173772,0.550351,0.747159,0.115209,-0.144531,-1.432526,0.309350,0.079108,-0.327085,0.631277,-0.481930,1.251270,0.158709,-0.307430,-0.476765,0.208309,-0.681993,...,0.109597,-0.746704,-0.611905,-1.246076,0.598580,-0.580133,-0.914450,0.119558,-0.130259,0.039278,126506,445713,292,270,0.924658,7,0,0,0,0,129,14,6,2,0,0,26920,1349,372,26261,0,3799,0,112496,0,29.612003,1,0.233439,19.955523,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,-0.176294,1.212596,0.335443,0.176997,

#SBERT NLI Embeddings

In [9]:
#For BERT CLS Embeddings Model
x_train, x_test, y_train, y_test = train_test_split(bert_cls_embeddings_df.iloc[:,:-1].values, bert_cls_embeddings_df.iloc[:,-1].values, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(403, 768)
(101, 768)
(403,)
(101,)


##SVM Classifier

In [10]:
# param_grid = {'C': [0.01, 0.1, 1, 10],
#               'gamma': [1,0.1],
#               'kernel': ['rbf','linear']}


# grid = GridSearchCV(SVC(),param_grid,verbose=10)
# grid.fit(x_train,y_train)
# print(grid.best_params_)

In [11]:
svm_clf = SVC(kernel='linear',C=0.01,gamma=1,verbose=True) ### Earlier C=1, gamma=0.01 gave 76% accuracy
svm_clf.fit(x_train,y_train)

[LibSVM]

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [43]:
#Dumping the SVM Model
joblib.dump(svm_clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings.pkl")
svm_clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings.pkl")

In [44]:
print("\n\nAccuracy report: ")
predict = svm_clf.predict(x_test)
print(classification_report(predict,y_test))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.65      0.77      0.71        39
           1       0.84      0.74      0.79        62

    accuracy                           0.75       101
   macro avg       0.74      0.76      0.75       101
weighted avg       0.77      0.75      0.76       101

(array([0, 1]), array([39, 62]))


##Random Forest Classifier

In [44]:
#Applying Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=400).fit(x_train, y_train)

In [45]:
#Dumping the Random Forest Model
joblib.dump(rfc,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings.pkl")
rfc = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings.pkl")

In [46]:
print("\n\nAccuracy report: ")
predict_test = rfc.predict(x_test)
print(classification_report(predict_test,y_test))
print(np.unique(predict_test,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.63      0.76      0.69        38
           1       0.84      0.73      0.78        63

    accuracy                           0.74       101
   macro avg       0.73      0.75      0.74       101
weighted avg       0.76      0.74      0.75       101

(array([0, 1]), array([38, 63]))


##One Vs All Classifier

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

clf = OneVsRestClassifier(SVC()).fit(x_train,y_train)

In [48]:
#Dumping the One Vs All Model
joblib.dump(clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings.pkl")
clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings.pkl")

In [49]:
outputs = clf.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.81      0.65      0.72        46
           1       0.75      0.87      0.81        55

    accuracy                           0.77       101
   macro avg       0.78      0.76      0.76       101
weighted avg       0.78      0.77      0.77       101



Model's Valid Distribution:  (array([0, 1]), array([37, 64]))


##Neural Networks

In [62]:
from sklearn.neural_network import MLPClassifier

### 800 --> 30 --> 10 --> 2

mlp = MLPClassifier(hidden_layer_sizes = (30,10),activation='relu',max_iter=10000)
mlp.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [63]:
#Dumping the Multi Perceptron Model
joblib.dump(mlp,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings.pkl")
mlp = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings.pkl")

In [64]:
outputs = mlp.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.75      0.59      0.66        46
           1       0.71      0.84      0.77        55

    accuracy                           0.72       101
   macro avg       0.73      0.71      0.71       101
weighted avg       0.73      0.72      0.72       101



Model's Valid Distribution:  (array([0, 1]), array([36, 65]))


#SBERT NLI Embeddings + Text

In [31]:
#For SBERT NLI Embeddings Model
x_train, x_test, y_train, y_test = train_test_split(bert_cls_embeddings_text_df.iloc[:,:-1].values, bert_cls_embeddings_text_df.iloc[:,-1].values, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(392, 778)
(99, 778)
(392,)
(99,)


##SVM Classifier

In [13]:
svm_clf = SVC(kernel='rbf',C=0.01,gamma=1,verbose=True) ### # on C=10, gamma=0.0001, we get arount 71
svm_clf.fit(x_train,y_train)

[LibSVM]

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [14]:
#Dumping the SVM Model
joblib.dump(svm_clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings_text.pkl")
svm_clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings_text.pkl")

In [15]:
print("\n\nAccuracy report: ")
predict = svm_clf.predict(x_test)
print(classification_report(predict,y_test))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.63      0.77        99

    accuracy                           0.63        99
   macro avg       0.50      0.31      0.39        99
weighted avg       1.00      0.63      0.77        99

(array([1]), array([99]))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Random Forest Classifier

In [32]:
#Applying Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=400).fit(x_train, y_train)

In [33]:
#Dumping the Random Forest Model
joblib.dump(rfc,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings_text.pkl")
rfc = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings_text.pkl")

In [34]:
print("\n\nAccuracy report: ")
predict_test = rfc.predict(x_test)
print(classification_report(predict_test,y_test))
print(np.unique(predict_test,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.62      0.70      0.66        33
           1       0.84      0.79      0.81        66

    accuracy                           0.76        99
   macro avg       0.73      0.74      0.73        99
weighted avg       0.77      0.76      0.76        99

(array([0, 1]), array([33, 66]))


##One Vs All Classifier

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

clf = OneVsRestClassifier(SVC()).fit(x_train,y_train)

In [16]:
#Dumping the One Vs All Model
joblib.dump(clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings_text.pkl")
clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings_text.pkl")

In [17]:
outputs = clf.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.50      0.19      0.27        37
           1       0.65      0.89      0.75        62

    accuracy                           0.63        99
   macro avg       0.57      0.54      0.51        99
weighted avg       0.59      0.63      0.57        99



Model's Valid Distribution:  (array([0, 1]), array([14, 85]))


##Neural Networks

In [18]:
from sklearn.neural_network import MLPClassifier

### 800 --> 30 --> 10 --> 2

mlp = MLPClassifier(hidden_layer_sizes = (30,10),activation='relu',max_iter=10000)
mlp.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [19]:
#Dumping the Multi Perceptron Model
joblib.dump(mlp,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings_text.pkl")
mlp = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings_text.pkl")

In [20]:
outputs = mlp.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.57      0.46      0.51        37
           1       0.71      0.79      0.75        62

    accuracy                           0.67        99
   macro avg       0.64      0.62      0.63        99
weighted avg       0.66      0.67      0.66        99



Model's Valid Distribution:  (array([0, 1]), array([30, 69]))


#SBERT + NLI + Embeddings + Text + User

In [75]:
#For SBERT NLI Embeddings Model
x_train, x_test, y_train, y_test = train_test_split(bert_cls_embeddings_text_user_df.iloc[:,:-1].values, bert_cls_embeddings_text_user_df.iloc[:,-1].values, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(368, 797)
(93, 797)
(368,)
(93,)


In [61]:
svm_clf = SVC(kernel='rbf',C=0.01,gamma=1,verbose=True)
svm_clf.fit(x_train,y_train)

[LibSVM]

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [62]:
#Dumping the SVM Model
joblib.dump(svm_clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings_text_user.pkl")
svm_clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/bert_cls_embeddings_text_user.pkl")

In [63]:
print("\n\nAccuracy report: ")
predict = svm_clf.predict(x_test)
print(classification_report(predict,y_test))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.58      0.73        93

    accuracy                           0.58        93
   macro avg       0.50      0.29      0.37        93
weighted avg       1.00      0.58      0.73        93

(array([1]), array([93]))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Random Forest Classifier

In [28]:
#Applying Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=400).fit(x_train, y_train)

In [29]:
#Dumping the Random Forest Model
joblib.dump(rfc,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings_text_user.pkl")
rfc = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_bert_cls_embeddings_text_user.pkl")

In [30]:
print("\n\nAccuracy report: ")
predict_test = rfc.predict(x_test)
print(classification_report(predict_test,y_test))
print(np.unique(predict_test,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.64      0.81      0.71        31
           1       0.89      0.77      0.83        62

    accuracy                           0.78        93
   macro avg       0.76      0.79      0.77        93
weighted avg       0.81      0.78      0.79        93

(array([0, 1]), array([31, 62]))


##One Vs All Classifier

In [76]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

clf = OneVsRestClassifier(SVC()).fit(x_train,y_train)

In [77]:
#Dumping the One Vs All Model
joblib.dump(clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings_text_user.pkl")
clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_bert_cls_embeddings_text_user.pkl")

In [78]:
outputs = clf.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.58      1.00      0.73        54

    accuracy                           0.58        93
   macro avg       0.29      0.50      0.37        93
weighted avg       0.34      0.58      0.43        93



Model's Valid Distribution:  (array([1]), array([93]))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Neural Networks

In [40]:
from sklearn.neural_network import MLPClassifier

### 800 --> 30 --> 10 --> 2

mlp = MLPClassifier(hidden_layer_sizes = (30,10),activation='relu',max_iter=10000)
mlp.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [41]:
#Dumping the Multi Perceptron Model
joblib.dump(mlp,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings_text_user.pkl")
mlp = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_bert_cls_embeddings_text_user.pkl")

In [42]:
outputs = mlp.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.54      0.38      0.45        39
           1       0.63      0.76      0.69        54

    accuracy                           0.60        93
   macro avg       0.58      0.57      0.57        93
weighted avg       0.59      0.60      0.59        93



Model's Valid Distribution:  (array([0, 1]), array([28, 65]))


#Just with User Features

In [79]:
#For BERT CLS Embeddings Model
x_train, x_test, y_train, y_test = train_test_split(bert_cls_embeddings_text_user_df.iloc[:,768:-1].values, bert_cls_embeddings_text_user_df.iloc[:,-1].values, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(368, 29)
(93, 29)
(368,)
(93,)


In [65]:
x_train[0]

array([3.02900000e+03, 1.69150000e+04, 2.65000000e+02, 2.64000000e+02,
       9.96226415e-01, 1.00000000e+01, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.55000000e+02, 1.60000000e+01,
       1.10000000e+01, 4.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.08440000e+04, 1.38900000e+03, 2.66000000e+02, 1.65260000e+04,
       1.00000000e+00, 3.23900000e+03, 1.00000000e+00, 6.29670000e+04,
       0.00000000e+00, 1.94402593e+01, 0.00000000e+00, 2.62454937e-01,
       7.80705544e+00])

##SVM 

In [66]:
svm_clf = SVC(kernel='rbf',C=0.01,gamma=1,verbose=True)
svm_clf.fit(x_train,y_train)

[LibSVM]

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [67]:
#Dumping the SVM Model
joblib.dump(svm_clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/svm_just_text_user.pkl")
svm_clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/svm_just_text_user.pkl")

In [68]:
print("\n\nAccuracy report: ")
predict = svm_clf.predict(x_test)
print(classification_report(predict,y_test))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.58      0.73        93

    accuracy                           0.58        93
   macro avg       0.50      0.29      0.37        93
weighted avg       1.00      0.58      0.73        93

(array([1]), array([93]))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Random Forest Classifier

In [69]:
#Applying Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=400).fit(x_train, y_train)

In [70]:
#Dumping the Random Forest Model
joblib.dump(rfc,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_just_text_user.pkl")
rfc = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Random Forest/rfc_just_text_user.pkl")

In [71]:
print("\n\nAccuracy report: ")
predict_test = rfc.predict(x_test)
print(classification_report(predict_test,y_test))
print(np.unique(predict_test,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

           0       0.59      0.68      0.63        34
           1       0.80      0.73      0.76        59

    accuracy                           0.71        93
   macro avg       0.69      0.70      0.70        93
weighted avg       0.72      0.71      0.71        93

(array([0, 1]), array([34, 59]))


##One Vs All Classifier

In [80]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

clf = OneVsRestClassifier(SVC()).fit(x_train,y_train)

In [81]:
#Dumping the One Vs All Model
joblib.dump(clf,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_just_text_user.pkl")
clf = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/OneVsAll/onevsall_just_text_user.pkl")

In [82]:
outputs = clf.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.58      1.00      0.73        54

    accuracy                           0.58        93
   macro avg       0.29      0.50      0.37        93
weighted avg       0.34      0.58      0.43        93



Model's Valid Distribution:  (array([1]), array([93]))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Neural Networks

In [83]:
from sklearn.neural_network import MLPClassifier

### 800 --> 30 --> 10 --> 2

mlp = MLPClassifier(hidden_layer_sizes = (5),activation='relu',max_iter=10000)
mlp.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=5, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [84]:
#Dumping the Multi Perceptron Model
joblib.dump(mlp,r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_just_text_user.pkl")
mlp = joblib.load(r"/gdrive/My Drive/2nd Sem/IBM/Covid Project/Models/Neural Network/nn_just_text_user.pkl")

In [85]:
outputs = mlp.predict(x_test)
print(classification_report(y_test,outputs))
print("\n\nModel's Valid Distribution: ",np.unique(outputs,return_counts = True))

              precision    recall  f1-score   support

           0       0.52      0.31      0.39        39
           1       0.61      0.80      0.69        54

    accuracy                           0.59        93
   macro avg       0.57      0.55      0.54        93
weighted avg       0.58      0.59      0.57        93



Model's Valid Distribution:  (array([0, 1]), array([23, 70]))
